In [1]:
import struct
import os
import numpy as np
from scipy import misc
import cv2
from random import randint
from IPython.core.debugger import Tracer

In [2]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

In [3]:
def image_label_list(data_dir):
    image_name_list = [ ]
    label_image_name_list = []
    label_name_list = [ ]
    for (dirpath, dirnames, filenames) in os.walk(data_dir + '/labels'):
        print "Training set has " + str(len(filenames)) + " files."
    for (dirpath, dirnames, filenames) in os.walk(data_dir + '/labels'):
        label_name_list.extend(map(lambda x: data_dir + '/labels/' + x, filenames))
        image_name_list.extend(map(lambda x: data_dir + '/images/' + x.replace('.dat', '.png'), filenames))
        label_image_name_list.extend(map(lambda x: data_dir + '/label_images/' + x.replace('.dat', '.png'), filenames))
        break
    return image_name_list, label_image_name_list, label_name_list

In [4]:
dir = "data_barcode/train"
crop_dir = "data_barcode/crop"
image_name_list, label_image_name_list, label_name_list = image_label_list(dir)

Training set has 300 files.


In [5]:
max_width = 0
max_height = 0
min_pad = 10
crop_w = 800
crop_h = 800
for label_idx, label_name in enumerate(label_name_list):
    if label_idx % 10 == 0:
        print label_idx
    image_name = image_name_list[label_idx]
    label_image_name = label_image_name_list[label_idx]
    image = misc.imread(image_name, mode='RGB')
    label_image = misc.imread(label_image_name, mode='RGB')
    height = image.shape[0]
    width = image.shape[1]
    f = open(label_name, "rb")
    binary_data = f.read()
    label = []
    for d in binary_data:
        label.append(struct.unpack('B', d)[0])
    label = np.array(label, dtype=np.uint8)
    label = label.reshape([height, width])
    label_img = 2 - label
#     print(label_img.shape)
#     plt.imshow(label_img, cmap='gray')
    contours = cv2.findContours(label_img, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cnt = contours[0]
    x, y, w, h = cv2.boundingRect(cnt)
    if w > max_width:
        max_width = w
    if h > max_height:
        max_height = h
    
    pad_w = crop_w - w
    pad_h = crop_h - h
    pad_left = randint(min_pad, pad_w - min_pad)
    x_1 = np.maximum(x - pad_left, 0)
    if x_1 + crop_w >= width:
        x_1 = width - crop_w - 1
    pad_top = randint(min_pad, pad_h - min_pad)
    y_1 = np.maximum(y - pad_top, 0)
    if y_1 + crop_h >= height:
        y_1 = height - crop_h - 1
    
    
    crop_img = image[y_1:y_1 + crop_h, x_1:x_1 + crop_w]
    crop_label_img = label_image[y_1:y_1 + crop_h, x_1:x_1 + crop_w]
    crop_label = label[y_1:y_1 + crop_h, x_1:x_1 + crop_w]
    misc.imsave(image_name.replace('train', 'crop'), crop_img)
    misc.imsave(label_image_name.replace('train', 'crop'), crop_label_img)
    crop_label.astype('uint8').tofile(label_name.replace('train', 'crop'))
#     plt.imshow(crop_label_img)
#     plt.imshow(crop_label, cmap='gray')
#     break

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
